In [63]:
# Note: Original saved in Tahir Sproj folder

# Deep Learning Libraries
# import torch
# import torch.nn as nn
# from torch.autograd import Variable
# import torch.utils.data as data

# Data Manipulation and Analysis
import numpy as np
# import pandas as pd

# Data Visualization
# import matplotlib
# import matplotlib.pyplot as plt

# File and System Interaction
# import os
# from pathlib import Path
# import torch.optim as optim


# Date and Time Handling
# import time
# import datetime

# Linear Algebra
# from torch import linalg as LA

# Neural Architecture
# try:
#     from torchinfo import summary
# except:
#     # %pip install torchinfo
#     from torchinfo import summary
from scipy.fftpack import dct

In [64]:
%load_ext autoreload
%autoreload 2

# from python_scripts import dataset_processing
# from python_scripts import architecture
# from python_scripts import revised_architecture
# from python_scripts import training
# from python_scripts import logs_and_results
# from python_scripts import img_pdf_compiler
from python_scripts import utils
from python_scripts import DUST_MC
from python_scripts import generate_synthetic_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Get relevant arguements for DUSTMC

M_train, M_Omega_train, M_test, M_Omega_test = generate_synthetic_data.generate_simple_gaussian_noise(150, 300, 10, 10, 2, 0.5, 3)

In [66]:
# Set the random seed for reproducibility
# np.random.seed(42)

rand_batch_idx = np.random.randint(0, M_Omega_train.shape[0])
X = M_Omega_train[rand_batch_idx, :, :]

# Initialize a m x n A measurement sensing matrix with random values from a uniform distribution between 0 and 1
A = np.random.uniform(0, 1, (100, 150)) # A is m x n. Since m << n and n is normally 150

# Initalize the Dictonary D which is n x d and d >> n by DCT. d is taken as 512
random_matrix = np.random.uniform(0, 1, (150, 512))

# Apply the Discrete Cosine Transform (DCT) to the matrix
D = dct(random_matrix, type = 2, norm = 'ortho')

# Initialize K (no. of iterations), and hyperparameters lambda_1, lambda_2, and c
K = 50
lambda_1 = 0.1
lambda_2 = 0.4
c = 1

In [67]:
A = np.eye(100, 150)
D = np.eye(150, 512)

In [68]:
def attention_based_algo2(A, D, X, K, lambda_1, lambda_2, c):
    d, T = D.shape[1], X.shape[1]
    H = np.zeros((d, T))
    
    for k in range(K):
        print(f'iteration {k + 1}/{K}.')
        for t in range(T):
            num, den = 0, 0

            for u in range(T):
                # print(u, H[:, t:t+1].T @ D.T @ D @ H[:, u:u+1])
                added = utils.beta_u(u, H) * np.exp(H[:, t:t+1].T @ D.T @ D @ H[:, u:u+1])
                den += added
                num += added * H[:, u:u+1]

            y = (lambda_2 / den) * num
            z = ((np.eye(d) - ((1/c) * D.T @ A.T @ A @ D)) @ y) + ((1/c) * D.T @ A.T @ A @ X[:, t:t+1])
            H[:, t:t+1] = utils.soft_thresholding(z, lambda_1 / c)
    
    return [D @ H[:, t] for t in range(T)]

attention_based_algo2(A, D, X, K, lambda_1, lambda_2, c)

iteration 1/50.
iteration 2/50.


/tmp/ipykernel_32217/3389217495.py:12: RuntimeWarning: overflow encountered in exp
  added = utils.beta_u(u, H) * np.exp(H[:, t:t+1].T @ D.T @ D @ H[:, u:u+1])
/tmp/ipykernel_32217/3389217495.py:12: RuntimeWarning: invalid value encountered in multiply
  added = utils.beta_u(u, H) * np.exp(H[:, t:t+1].T @ D.T @ D @ H[:, u:u+1])
/tmp/ipykernel_32217/3389217495.py:14: RuntimeWarning: invalid value encountered in multiply
  num += added * H[:, u:u+1]


iteration 3/50.
iteration 4/50.
iteration 5/50.


KeyboardInterrupt: 

In [69]:
# Now run the attention based algo on X to get its recovered/reconstructed form S

S = DUST_MC.attention_based_algo(A, D, X, K, lambda_1, lambda_2, c)

/home/nehal/GitHubWorkspace/ConvHuberMC-Net/python_scripts/DUST_MC.py:39: RuntimeWarning: invalid value encountered in scalar multiply
  G_numerator += beta_u * exp_value * h_u_prev
/home/nehal/GitHubWorkspace/ConvHuberMC-Net/python_scripts/DUST_MC.py:40: RuntimeWarning: invalid value encountered in scalar multiply
  G_denominator += beta_u * exp_value


KeyboardInterrupt: 

In [ ]:
# Now measure loss between reconstructed and groundtruth

groundtruth = M_train[rand_batch_idx, :, :]

print(f'The squared L2 norm loss is: {utils.compute_squared_l2_norm_loss(S, groundtruth)}')